In [1]:
import string

def tokenize(lines):
    lines = [line.rstrip() for line in lines[:nr_used_sentences]] #remove \n
    words = [line.split() for line in lines] #split by words
    unpunctuated = str.maketrans('', '', string.punctuation) #remove punctuation & make Lowercase
    lower = [[word.translate(unpunctuated).lower() for word in sentence] for sentence in words]
    return lower

def read_in_corpus(nr_used_sentences, corpus_path):
    filename_e = "BU_en.txt"
    filename_f = "BU_tr.txt"

    e_file = open(corpus_path + filename_e, "r", encoding='utf-8-sig')
    e_lines = e_file.readlines()
    f_file = open(corpus_path + filename_f, "r", encoding='utf-8-sig')
    f_lines = f_file.readlines()

    f = tokenize(f_lines)
    e = tokenize(e_lines)
    return e, f

nr_used_sentences = 30 #of 688670 lines
path = "./Paralel Corpus/"
e, f = read_in_corpus(nr_used_sentences, path)

In [2]:
import numpy as np

max_steps = 50
training_split = 0.9

def split_dataset(dataset, training_split):
    np.random.shuffle(dataset)
    nr_train_samples = int(training_split*len(dataset))
    return dataset[:nr_train_samples], dataset[nr_train_samples:]

e_train, e_test = split_dataset(e, training_split)
f_train, f_test = split_dataset(f, training_split)
e_train = [['house','the','the'],['the','the','book'],['a','book']]
f_train = [['das','Haus'],['das','Buch'],['ein','Buch']]

e_words = set([item for sublist in e_train for item in sublist])
f_words = set([item for sublist in f_train for item in sublist])
    
#test if the two sets are equivalent
def is_converged(t, last_t):
    return t == last_t

In [3]:
import copy
#learning translation probabilitity distributions from sentence-aligned parallel text
#expectation maximization algorithm
#Input: set of sentence pairs (e,f) 
#Output: translation prob. t(e|f)
#S.91 Figure 4.3
def EM_IBM_Model_1 (e_set,f_set):
    #sets of distinct words
    e_words = set([item for sublist in e_set for item in sublist])
    f_words = set([item for sublist in f_set for item in sublist])
    #initialize t(e|f) with uniform distribution
    t = {}#{key: {key2: 1/len(e_words) for key2 in f_words} for key in e_words} #make code faster
    for k in range (0, len(e_set)):
        e = e_set[k]
        f = f_set[k]
        for e_j in e:
            for f_i in f:
                t[(e_j,f_i)] = 1/len(e_words) #TODO is this right? divide by 4
    last_t = {0:1} #to fail first comparison
    step = 0
    #iterate until convergence
    while not(is_converged(t, last_t)) and step < max_steps:
        #initialize
        count = {(e,f): 0 for f in f_words for e in e_words}
        total = {f: 0 for f in f_words}
        for k in range (0, len(e_set)):#works cause number lines same
            e = e_set[k]
            f = f_set[k]
            #compute normalization
            #s_total = {e: 0 for e in e_words}
            s_total = {}
            for e_j in e:
                s_total[e_j] = 0
                for f_i in f:
                    s_total[e_j] += t[(e_j,f_i)]
            #collect counts
            for f_i in f:
                for e_j in e:
                    count[(e_j,f_i)] += t[(e_j,f_i)] / s_total[e_j]
                    total[f_i] += t[(e_j,f_i)] / s_total[e_j]
        #estimate probabilities
        last_t = copy.deepcopy(t)
        for f_i in f_words:
            for e_j in e_words:
                if (e_j,f_i) in t:
                    t[(e_j,f_i)] = count[(e_j,f_i)] / total[f_i] 
        step += 1
        #print(last_t)
        #print('---')
    return t
print(e_train)
t = EM_IBM_Model_1(e_train,f_train)
#print(t)

#compare t table to IBM Model 1 implementation of nltk library
import nltk
from nltk.translate import AlignedSent, IBMModel1
bitext = []
bitext.append(AlignedSent(['das', 'Haus'], ['house','the','the']))
bitext.append(AlignedSent(['das', 'Buch'], ['the', 'the','book']))
bitext.append(AlignedSent(['ein', 'Buch'], ['a', 'book']))
ibm1 = IBMModel1(bitext, 50)
for f_i in f_words:
    for e_j in e_words:
        if (e_j,f_i) in t:
            print(t[(e_j,f_i)], ibm1.translation_table[f_i][e_j])

[['house', 'the', 'the'], ['the', 'the', 'book'], ['a', 'book']]
0.333333333333345 0.011414945271378608
0.6666666666666549 0.9999999999994895
0.014902246322072639 1e-12
0.9850977536779216 0.9961746545566007
5.696228382142683e-15 1e-12
1.0 0.9885850547281109
1.0063233751843367e-25 1e-12
2.9688175631860936e-18 1e-12
0.017831796971375664 0.0038253454429096182
0.9821682030286244 0.9999999999994867


In [14]:
#learning translation probabilitity distributions from sentence-aligned parallel text
#expectation maximization algorithm
#Input: set of sentence pairs (e,f) 
#Output: translation prob. t (lexical translation) and a (alignment)
#S.99 Figure 4.7
#TODO e = [None] + e
def EM_IBM_Model_2 (e_set,f_set):
    #initialize t(e|f) with IBM Model 1
    t = EM_IBM_Model_1(e_set,f_set)
    #initialize a(i|j, l_e, l_f)
    e_lengths = [len(e) for e in e_set]
    f_lengths = [len(f) for f in f_set]
    a = {}
    for k in range (0, len(e_set)): #for every sentence
        e = e_set[k]
        f = f_set[k]
        l_e = len(e)
        l_f = len(f)
        for j in range(0, l_e):
            for i in range(0, l_f):
                a[(i,j,l_e,l_f)] = 1/(l_f + 1)
    #sets of distinct words
    e_words = list(set([item for sublist in e_set for item in sublist]))
    f_words = list(set([item for sublist in f_set for item in sublist]))
    last_t = {} #to fail first comparison
    step = 0
    #iterate until convergence
    while not (is_converged(t, last_t)) and step < max_steps:
        #initialize
        count = {(e,f): 0 for f in f_words for e in e_words}
        total = {f: 0 for f in f_words}
        total_a = {}
        count_a = {}
        for l_e in e_lengths:
            for l_f in f_lengths:
                for j in range(0,l_e):
                    total_a[(j,l_e,l_f)] = 0
                    for i in range(0,l_f):
                        count_a[(i,j,l_e,l_f)] = 0
        for k in range(0, len(e_set)):
            e = e_set[k]
            f = f_set[k]
            l_e = len(e)
            l_f = len(f)
            #compute normalization
            s_total = {}
            for j in range(0, l_e):
                e_j = e[j]
                s_total[e_j] = 0
                for i in range(0, l_f):
                    f_i = f[i]
                    s_total[e_j] += t[(e_j,f_i)] * a[(i,j,l_e,l_f)]
            #collect counts
            for j in range(0, l_e):
                for i in range(0, l_f):
                    e_j = e[j]
                    f_i = f[i]
                    c = t[(e_j,f_i)] * a[(i,j,l_e,l_f)] / s_total[e_j]
                    count[(e_j,f_i)] += c
                    total[f_i] += c
                    count_a[(i,j,l_e,l_f)] += c
                    total_a[(j,l_e,l_f)] += c
        #estimate probabilities
        last_t = copy.deepcopy(t)
        for f_i in f_words:
            for e_j in e_words:
                if (e_j,f_i) in t:
                    t[(e_j,f_i)] = count[(e_j,f_i)] / total[f_i]
        for l_e in e_lengths:
            for l_f in f_lengths:
                for i in range(0,l_f):
                    for j in range(0,l_e):
                        if (i,j,l_e,l_f) in a:
                            a[(i,j,l_e,l_f)] = count_a[(i,j,l_e,l_f)] / total_a[(j,l_e,l_f)]    
        step += 1
    return t, a

#compare t and a table to IBM Model 2 implementation of nltk library
def compare_to_nltk(t,a,src,tar):
    import nltk
    from nltk.translate import AlignedSent, IBMModel2
    bitext = []
    bitext.append(AlignedSent(['das', 'Haus'], ['house','the','the']))
    bitext.append(AlignedSent(['das', 'Buch'], ['the', 'the','book']))
    bitext.append(AlignedSent(['ein', 'Buch'], ['a', 'book']))
    ibm2 = IBMModel2(bitext, 49)

    for f_i in f_words:
        for e_j in e_words:
            if (e_j,f_i) in t:
                bool = (ibm2.translation_table[f_i][e_j] > 0.7) == (t[(e_j,f_i)] > 0.7)
                if bool == False: print('t', t[(e_j,f_i)], ibm2.translation_table[f_i][e_j])

    e_lengths = [len(e) for e in src]
    f_lengths = [len(f) for f in tar]
    for l_e in e_lengths:
        for l_f in f_lengths:
            for i in range(0,l_f):
                for j in range(0,l_e):
                    if (i,j,l_e,l_f) in a:
                        bool = (a[(i,j,l_e,l_f)] > 0.7) == (ibm2.alignment_table[j][i][l_f][l_e] > 0.7)
                        if bool == False: 
                            #print('a', a[(i,j,l_e,l_f)], ibm2.alignment_table[j][i][l_f][l_e])
                            print('i',i,' j',j,' l_e',l_e,' l_f',l_f)
                        #else:
                            #print('true','i',i,' j',j,' l_e',l_e,' l_f',l_f)
#TODO my a is 1 when it should be low :(
#TODO wrong for other direction

t, a = EM_IBM_Model_2(e_train,f_train)
compare_to_nltk(t,a,e_train,f_train)
#print('t',t)
#print('a',a)
print('------')
t, a = EM_IBM_Model_2(f_train,e_train)
compare_to_nltk(t,a,f_train,e_train)

i 0  j 1  l_e 3  l_f 2
i 1  j 0  l_e 3  l_f 2
i 0  j 1  l_e 3  l_f 2
i 1  j 0  l_e 3  l_f 2
i 0  j 1  l_e 3  l_f 2
i 1  j 0  l_e 3  l_f 2
i 0  j 1  l_e 3  l_f 2
i 1  j 0  l_e 3  l_f 2
i 0  j 1  l_e 3  l_f 2
i 1  j 0  l_e 3  l_f 2
i 0  j 1  l_e 3  l_f 2
i 1  j 0  l_e 3  l_f 2
i 0  j 0  l_e 2  l_f 2
i 0  j 0  l_e 2  l_f 2
i 0  j 0  l_e 2  l_f 2
------
i 1  j 0  l_e 2  l_f 3
i 1  j 0  l_e 2  l_f 3
i 0  j 0  l_e 2  l_f 2
i 1  j 0  l_e 2  l_f 3
i 1  j 0  l_e 2  l_f 3
i 0  j 0  l_e 2  l_f 2
i 1  j 0  l_e 2  l_f 3
i 1  j 0  l_e 2  l_f 3
i 0  j 0  l_e 2  l_f 2


In [62]:
#Testing

#returns most likely aligned input word position i for a given position j in output sentence
def a_fct(j,e,f,t):
    e_j = e[j]
    max_t = 0
    max_i = None
    for i in range(0,len(f)):
        f_i = f[i]
        if (e_j,f_i) in t and t[(e_j,f_i)] > max_t: 
            max_t = t[(e_j,f_i)]
            max_i = i
    return max_i
    #TODO return max (a*t)!!!!

#p(e|f) for IBM Model 1
#S.90 Figure 4.10
def prob_e_given_f_1 (e, f, epsilon, t):
    prod = 1
    for e_j in e: #TODO starts at 1 not at 0!?
        sum = 0
        for f_i in f:
            if (e_j,f_i) in t:
                sum += t[(e_j,f_i)]
        prod *= sum
    return ( epsilon / len(f)**len(e) ) * prod
    #without +1 for NONE

#p(e|f) for IBM Model 2
#S.98 Figure 4.26
#TODO we can leave out for i, cause i isnt used?
#TODO but maybe we cant cause the sum changes eventhough!
def prob_e_given_f_2 (e, f, epsilon, t, a):
    l_e = len(e)
    l_f = len(f)
    prod = 1
    for j in range(0, l_e): #TODO starts at 1 not at 0!
        e_j = e[j]
        sum = 0
        for i in range(0, l_f): 
            f_i = f[i]
            if (e_j,f_i) in t:
                f_aj = f[a_fct(j,e,f,t)]
                sum += t[(e_j,f_aj)]*a[(a_fct(j,e,f,t),j,l_e,l_f)]
        prod *= sum
    return epsilon * prod
#when removing += it's 1 and not 4 anymore which is good - so maybe dont sum over i?

for i in range(0,len(f_test)):
    f = f_test[i]
    e = e_test[i]
    #arg max_e p(e)p(f|e) decoding manually #TODO how???
    #calculate p(e|f)
    #print(e, prob_e_given_f(e, f, 0.001, t))
    
f = ['das','Buch']
e = ['the','book']
print('p(',e,'|',f,') =',prob_e_given_f_1(e, f, 1, t))
print('a: p(',e,'|',f,') =',prob_e_given_f_2(e, f, 1, t, a))
e = ['kgefw','fek']
print('p(',e,'|',f,') =',prob_e_given_f_1(e, f, 1, t))
print('a: p(',e,'|',f,') =',prob_e_given_f_2(e, f, 1, t, a))

p( ['the', 'book'] | ['das', 'Buch'] ) = 0.0
a: p( ['the', 'book'] | ['das', 'Buch'] ) = 0
p( ['kgefw', 'fek'] | ['das', 'Buch'] ) = 0.0
a: p( ['kgefw', 'fek'] | ['das', 'Buch'] ) = 0


In [17]:
#Phrase-based model

#Sentence Alignments in both directions with Viterbi algorithm using IBM Model 2
#While it is true that during the EM training fractional counts are
#collected over a probability distribution of possible alignments, one
#word alignment sticks out: the most probable alignment, also called the
#Viterbi alignment
#> output the Viterbi alignment of the last iteration
#> run IBM model training in both directions

#Decoding

#TODO I have to use ibm model 2!!!!
#Viterbi alignment
#find the most probable alignment for a given translation pair
#f2e = ibmmodel2.viterbi_alignment(es, fs, *f2e_train).items()
#e2f = ibmmodel2.viterbi_alignment(fs, es, *e2f_train).items()
#align = alignment(es, fs, e2f, f2e)  # symmetrized alignment
"""""
def viterbi_alignment(es, fs, t, a):
    '''
    return
        dictionary
            e in es -> f in fs
    '''
    max_a = collections.defaultdict(float)
    l_e = len(es)
    l_f = len(fs)
    for (j, e) in enumerate(es, 1):
        current_max = (0, -1)
        for (i, f) in enumerate(fs, 1):
            val = t[(e, f)] * a[(i, j, l_e, l_f)]
            # select the first one among the maximum candidates
            if current_max[1] < val:
                current_max = (i, val)
        max_a[j] = current_max[0]
    return max_a
"""""
#a_hat = arg max_a p(e_j,f_i)
def align(e_set, f_set, t):
    all_aligned = {}
    for sentence in range(0,len(e_set)):
        e = e_set[sentence]
        f = f_set[sentence]
        aligned = np.zeros((len(e),len(f)))
        for i in range(0, len(f)): 
            f_i = f[i]
            best_prob = 0
            best_j = 0
            for j in range(0, len(e)):
                e_j = e[j]
                if (e_j,f_i) in t and t[(e_j,f_i)] > best_prob:
                    best_prob = t[(e_j,f_i)]
                    best_j = j
            aligned[best_j][i] = 1
        all_aligned[sentence] = aligned
    return all_aligned

#IBM Models for word alignment
t_e2f = EM_IBM_Model_1(e_train,f_train)
all_aligned_e2f = align(e_train,f_train,t_e2f)
t_f2e = EM_IBM_Model_1(f_train,e_train)
all_aligned_f2e = align(f_train,e_train,t_f2e)
#transpose so that alignments are same shape
all_aligned_f2e = [np.transpose(f2e) for f2e in list(all_aligned_f2e.values())]
for sentence in range(0,len(e_train)):
    print(e_train[sentence], f_train[sentence])
    print(all_aligned_e2f[sentence])
    print(all_aligned_f2e[sentence])
    #x foreign, y English

['house', 'the', 'the'] ['das', 'Haus']
[[0. 1.]
 [1. 0.]
 [0. 0.]]
[[0. 1.]
 [1. 0.]
 [1. 0.]]
['the', 'the', 'book'] ['das', 'Buch']
[[1. 0.]
 [0. 0.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [0. 1.]]
['a', 'book'] ['ein', 'Buch']
[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]


In [18]:
#Symmetrization of alignment
#for every sentence a alignment matrix esentence - fsentence
#0 if not aligned, 1 if aligned 
#2 directions -> make intersection
#make union, neighbour logic
#for each sentence combine the two alignments using the word alignment algorithm
#>merged by taking the intersection or the union of alignment points of each alignment
#The algorithm starts with the intersection of the alignments. In
#the growing step, neighboring alignment points that are in the union but
#not in the intersection are added. In the final step, alignment points for
#words that are still unaligned are added.

def intersect(a,b):
    intersect = np.zeros(np.shape(a))
    for j in range(0,np.shape(a)[0]):
        for i in range(0,np.shape(a)[1]):
            intersect[j][i] = 1 if (a[j][i]==1 and b[j][i]==1) else 0
    return intersect

def union(a,b):
    union = np.zeros(np.shape(a))
    for j in range(0,np.shape(a)[0]):
        for i in range(0,np.shape(a)[1]):
            union[j][i] = 1 if (a[j][i]==1 or b[j][i]==1) else 0
    return union

def aligned_e(j,a):
    return np.sum(a[j]) != 0

def aligned_f(i,a):
    return np.sum([a[j][i] for j in range(0,np.shape(a)[0])]) != 0

def aligned(j, i, a):
    return a[j][i] == 1

def neighbour_point(j, i):
    neighbouring = [(-1,0),(0,-1),(1,0),(0,1),(-1,-1),(-1,1),(1,-1),(1,1)]
    return [(a+j, b+i) for (a,b) in neighbouring]

def grow_diag_final(e2f, f2e):
    a = intersect(e2f,f2e)
    union_alignment = union(e2f,f2e)
    last_a = []
    #iterate until convergence
    while not np.array_equal(last_a,a):
        last_a = copy.deepcopy(a)
        for j in range(0,np.shape(e2f)[0]):
            for i in range(0,np.shape(e2f)[1]):
                if aligned(j, i, a):
                    for (j_new, i_new) in neighbour_point(j, i):
                        if j_new in a and i_new in a[j_new]:
                            if (not aligned_e(j_new, a) or not aligned_f(i_new, a)) and (j_new, i_new) in union_alignment:
                                a[j_new][i_new] = 1
    for j_new in range(0,np.shape(a)[0]):
        for i_new in range(0,np.shape(a)[1]):
            if (not aligned_e(j_new,a) or not aligned_f(i_new,a)) and (j_new, i_new) in union_alignment: 
                a[j_new][i_new] = 1
    return a

#call for every sentence pair
all_alignments = {}
for index in range(0,len(e_train)):
    e2f = all_aligned_e2f[index]
    f2e = all_aligned_f2e[index]
    alignment = grow_diag_final(e2f, f2e)
    print(alignment)
    all_alignments[index] = alignment

[[0. 1.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]


In [19]:
#Phrase extraction from symmetrized alignment table
#phrase extraction algorithm Use a suitable threshold for maximum phrase length and 
#estimate the phrase translation probabilities. 
#Phrase extraction algorithm: 
#For each English phrase estart .. eend,
#the minimal phrase of aligned foreign words is identified fstart .. fend. Words in
#the foreign phrase are not allowed to be aligned with English words outside the
#English phrase. This pair of phrases is added, along with additional phrase pairs
#that include additional unaligned foreign words at the edge of the foreign phrase.
#Input: word alignment A for sentence pair (e,f)
#Output: set of phrase pairs BP

#TODO max phrase length
def extract(f, f_start, f_end, e, e_start, e_end, a):
    #check if at least one alignment point
    if f_end < 0:
        return []
    #check if alignment points violate consistency
    for j in range(0, np.shape(a)[0]):
        for i in range(0, np.shape(a)[1]):
            if a[j][i] == 1:
                if (f_start <= i <= f_end) and (j < e_start or j > e_end):
                    return []
    #add pharse pairs (incl. additional unaligned f)
    E = []
    f_s = f_start
    first1 = True
    while not aligned_f(f_s,a) or first1:
        first1 = False
        f_e = f_end
        first2 = True
        while not aligned_f(f_e,a) or first2:
            first2 = False
            E.append((e[e_start:e_end+1], f[f_s:f_e+1]))
            f_e += 1
            if f_e >= np.shape(a)[1]: break
        f_s -= 1
        if f_s < 0: break
    return E

e_train = [['house','the','the'],['the','the','book'],['a','book']]
f_train = [['das','Haus'],['das','Buch'],['ein','Buch']]
"""""
e_train = [['michael','assumes','that','he','will','stay','in','the','house']]
f_train = [['michael','geht','davon','aus',',','dass','er','im','haus','bleibt']]
all_alignments = {0:[[1,0,0,0,0,0,0,0,0,0],
                   [0,1,1,1,0,0,0,0,0,0],
                   [0,0,0,0,0,1,0,0,0,0],
                   [0,0,0,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,0,0,0,1],
                   [0,0,0,0,0,0,0,0,0,1],
                   [0,0,0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,0,0,1,0]]}
"""""

def phrase_extraction(e_set,f_set, all_alignments):
    all_BP = []
    for index in all_alignments:
        alignment = all_alignments[index]
        e = e_set[index]
        f = f_set[index]
        BP = []
        for e_start in range(0,len(e)):
            for e_end in range(e_start,len(e)):
                #find the minimally matching foreign phrase
                (f_start, f_end) = (len(f)-1, -1)
                for j in range(0, np.shape(alignment)[0]):
                    for i in range(0, np.shape(alignment)[1]):
                        if alignment[j][i] == 1:
                            if e_start <= j <= e_end:
                                f_start = min(i, f_start)
                                f_end = max(i, f_end)
                BP.append(extract(f, f_start, f_end, e, e_start, e_end, alignment))
        all_BP.append([item for sublist in BP for item in sublist])
    return all_BP

all_BP = phrase_extraction(e_train, f_train, all_alignments)
#for BP in all_BP:
print(all_BP)

[[(['house'], ['Haus']), (['house', 'the', 'the'], ['das', 'Haus']), (['the', 'the'], ['das'])], [(['the', 'the'], ['das']), (['the', 'the', 'book'], ['das', 'Buch']), (['book'], ['Buch'])], [(['a'], ['ein']), (['a', 'book'], ['ein', 'Buch']), (['book'], ['Buch'])]]


In [73]:
#probabilistic phrase translation table
#For each sentence pair, we extract a number of phrase pairs.
#Then, we count in how many sentence pairs a particular phrase pair is
#extracted and store this number in count(e,f).
all_BP_flat = [item for sublist in all_BP for item in sublist]
count={}
for (a,b) in all_BP_flat:
    #key = (frozenset(a),frozenset(b))
    key = str((a,b))
    count[key] = count.get(key,0) + 1
print(count)

#Scoring Phrase Translations
#the translation probability φ(f|e) is estimated by the relative frequency
def φ(e, f):
    key = str((e,f))
    sum = 0
    for f_i in f:
        if str((e,[f_i])) in count:
            sum +=count[str((e,[f_i]))]
    if str((e,f)) in count:
        return count[key] / sum
    else: return 0

print(φ(['house'],['Haus']))
#print(φ(['the','house'],['das','Haus']))
#TODO translation probability is not working for phrases containing several words
#maybe other data structure ['the','house']:[['das','Haus'],['ein','Haus']]
#faster sometimes but slower if translation in other direction

#TODO For the estimation of the phrase translation probabilities, not all
#phrase pairs have to be loaded into memory. It is possible to efficiently
#estimate the probability distribution by storing and sorting the extracted
#phrases on disk. Similarly, when using the translation table for the translation of a single sentence, only a small fraction of it is needed and may
#be loaded on demand.
#think about saving stuff in file
#• Phrase translation table typically bigger than corpus
#... even with limits on phrase lengths (e.g., max 7 words)
#→ Too big to store in memory?
#• Solution for training
#– extract to disk, sort, construct for one source phrase at a time
#• Solutions for decoding
#– on-disk data structures with index for quick look-ups
#– suffix arrays to create phrase pairs on demand
#TODO use numpy for everything

#phrase-based statistical machine translation model
#• the phrase translation table φ(f |e);
#• the reordering model d;
#• the language model pLM(e).
#e_best = argmax_e prod_i φ(f|e) * d(start_i - end_i_min_1 - 1) * prod_i PLM(e_i|e_i_min_1)

#distance based reordering model d
#This model gives a cost linear to the reordering distance. For instance, skipping over two words costs twice as much as skipping over one word.
#<source-phrase> & <target-phrase> : <weight-1> <weight-2> ... <weight-k> for k of 6 / 8
def d(x):
    return 0#alpha**|x| #TODO i dont get it

#bigram language model #TODO i dont get it
#<probability> <word-1><word-2>  <back-off-weight>
#eg -0.2359 When will 0.1011
#test the model using the test data. 
#for a source sentence, you can generate some example target sentences (word sequences) 
#and phrases “manually”; i.e. you will do the decoding process manually. 
#That is, for a source sentence, generate a possible target sentence,
#divide both sentences into phrases as you wish, and assume that you know the phrase
#correspondences. Then, calculate the probability of the target sentence given the source
#sentence using the p(f|e)*p LM (e) equation (i.e. using the standard model with three
#components). 

#You can train a simple bigram language model and a reordering model.

#Translation model - provides phrases in the source language with learned possible target language 
#translations and the probabilities thereof.
#Reordering model - stores information about probable translation orders of the phrases within the 
#source text, based on the observed source and target phrases and their alignments.
#Language model - reflects the likelihood of this or that phrase in the target language to occur. 
#In other words, it is used to evaluate the obtained translation for being "sound" in the target language.
#Note that, the language model is typically learned from a different corpus in a target language.

#With these three models at hand one can perform decoding, which is a synonym to a translation process. 
#SMT decoding is performed by exploring the state space of all possible translations and reordering of the 
#source language phrases within one sentence. The purpose of decoding, as indicated by the maximization 
#procedure at the bottom of the figure above, is to find a translation with the largest possible probability.

{"(['house'], ['Haus'])": 1, "(['house', 'the', 'the'], ['das', 'Haus'])": 1, "(['the', 'the'], ['das'])": 2, "(['the', 'the', 'book'], ['das', 'Buch'])": 1, "(['book'], ['Buch'])": 2, "(['a'], ['ein'])": 1, "(['a', 'book'], ['ein', 'Buch'])": 1}
0


In [ ]:
#QUESTIONS

#IBM Model 2: Do I calculate a correctly?
#> compare with other implementations
#> NOOOOO a is wrong :/

#p(e|f) for IBM Model 2: Gives result 4 should be 1?
#> compare with other implementations

#TODO did I have to implement all that? 
#what exactly do i have to do? first train, then do what with t?
#decoding = calculating max p(e|f)?

#how to output the Viterbi alignment of the last iteration IBM 2?

#how to implement distance based reordering model?

#how to implement bigram language model?